In [1]:
from pybliometrics.scopus import AbstractRetrieval, ScopusSearch
from tqdm import tqdm
import pandas as pd

In [2]:
# X 데이터 수집을 위한 기준 데이터 생성
with open("rsc/preparation_data/AI_target_eids_2005-2010.txt", "r") as f:
     ai_paper_eids = [eid.strip() for eid in f.readlines()]
print("AI 논문 개수:", len(ai_paper_eids))

with open("rsc/preparation_data/automotive_target_eids_2005-2010.txt", "r") as f:
     ae_paper_eids = [eid.strip() for eid in f.readlines()]
print("AE 논문 개수:", len(ae_paper_eids))

df_ai = pd.DataFrame({'eid':ai_paper_eids})
df_ai['div'] = 'AI'
df_ae = pd.DataFrame({'eid':ae_paper_eids})
df_ae['div'] = 'AE'

df = pd.concat([df_ai, df_ae]).reset_index(drop=True)

AI 논문 개수: 65470
AE 논문 개수: 39347


coverDate, title, description, abstract, 

In [6]:
ab_objs = []
for eid in tqdm(df.loc[:, 'eid']):
    try:
        ab = AbstractRetrieval(eid, view='FULL')
        ab_objs.append(ab)
    except:
        pass

100%|██████████| 104817/104817 [01:00<00:00, 1725.86it/s] 


In [9]:
AbstractRetrieval('2-s2.0-33750451262', view='FULL')

IndexError: list index out of range

2-s2.0-55549138016 의 아이디가 retrieval 되지 않음 => 확인 필요

In [104]:
title = ab.title # Final
abstract = ab.abstract # Final
author_keywords = ab.authkeywords # Final
index_keywords = ab.idxterms
published_year = ab.coverDate[:4] # Final
language = ab.language # 영어 기반의 버트 모델이므로 영어 이외의 문서는 제거
affiliation_ids = ";".join([str(affiliation_info.id) for affiliation_info in ab.affiliation]) # 소속 기관의 연구자 수, 총 논문 수를 구할 ID
author_ids = ";".join([str(author_info.auid) for author_info in ab.authors])
journal_issn = ab.issn

In [106]:
ab.references

[Reference(position='1', id='0003728220', doi=None, title=None, authors='Nilsson, N.J.', authors_auid=None, authors_affiliationid=None, sourcetitle='Learning Machines', publicationyear='1965', volume=None, issue=None, first=None, last=None, citedbycount=None, type=None, text='McGraw-Hill', fulltext='N.J. Nilsson, Learning Machines. McGraw-Hill, 1965.'),
 Reference(position='2', id='0003450542', doi=None, title=None, authors='Vapnik, V.', authors_auid=None, authors_affiliationid=None, sourcetitle='The Nature of Statistical Learning Theory', publicationyear='1995', volume=None, issue=None, first=None, last=None, citedbycount=None, type=None, text='Springer', fulltext='V. Vapnik, The Nature of Statistical Learning Theory. Springer, 1995.'),
 Reference(position='3', id=None, doi=None, title=None, authors='', authors_auid=None, authors_affiliationid=None, sourcetitle=None, publicationyear='2009', volume=None, issue=None, first=None, last=None, citedbycount=None, type=None, text='OSU-SVM-TOO

In [91]:
ab.authors

[Author(auid=22634035000, indexed_name='Escalera S.', surname='Escalera', given_name='Sergio', affiliation='60104040'),
 Author(auid=13005298900, indexed_name='Pujol O.', surname='Pujol', given_name='Oriol', affiliation='60104040'),
 Author(auid=56208405900, indexed_name='Radeva P.', surname='Radeva', given_name='Petia', affiliation='60104040')]

In [64]:
s.results

[Document(eid='2-s2.0-84871632945', doi='10.1109/TPAMI.2010.8', pii=None, pubmed_id=None, title='The 30th Anniversary of the IEEE Transactions on Pattern Analysis and Machine Intelligence', subtype='ed', subtypeDescription='Editorial', creator='Zabih R.', afid=None, affilname=None, affiliation_city=None, affiliation_country=None, author_count='1', author_names='Zabih, Ramin', author_ids='6701366172', author_afids=None, coverDate='2010-12-01', coverDisplayDate='2010', publicationName='IEEE Transactions on Pattern Analysis and Machine Intelligence', issn='01628828', source_id='24254', eIssn=None, aggregationType='Journal', volume='32', issueIdentifier='1', article_number='5339306', pageRange='1', description=None, authkeywords=None, citedby_count=0, openaccess=0, fund_acr=None, fund_no='undefined', fund_sponsor=None)]